In [1]:
import pandas as pd


In [2]:
# Extract data from sensor database:
# \copy (select date(observed_at),classroom_id,count(observed_at) as ob_count from radio_observation group by classroom_id, date(observed_at) order by date(observed_at)) To 'daily_records.csv' With CSV HEADER;
df = pd.read_csv('daily_records.csv',index_col=0,parse_dates=True)
df.head()

,classroom_id,ob_count
date,,
1952-06-23,735,16
1952-06-23,725,1
1970-01-08,735,2
1970-07-22,725,592
1974-04-10,725,16


In [3]:
df.classroom_id.unique()

array([ 735,  725,  676, 1289])

In [4]:
total_days = 0
data_days = 0
for classroom in df.classroom_id.unique():
    if classroom == 735:  # Skip sandbox classroom
        continue
    room_data = df[df.classroom_id == classroom]
    del room_data['classroom_id']
    room_data = room_data[room_data.ob_count > 1000]
    start_date = room_data.index[0]
    end_date = room_data.index[-1]
    all_dates = pd.date_range(start_date, end_date)
    week_days = all_dates[all_dates.weekday < 5]
    room_data = room_data.reindex(week_days, fill_value=0)
    total_days += len(room_data)
    classroom_data_days = len(room_data[room_data.ob_count > 0])
    data_days += classroom_data_days
    pct = classroom_data_days / len(room_data) * 100
    print("Classroom %d = %d/%d, %f%%" % (classroom, len(room_data), classroom_data_days, pct))
    #print(room_data.head(20))
    
print("Total: %d/%d = %f%%" % (data_days, total_days, float(data_days)/total_days * 100))
    

Classroom 725 = 65/37, 56.923077%
Classroom 676 = 50/17, 34.000000%
Classroom 1289 = 38/7, 18.421053%
Total: 61/153 = 39.869281%
